# RAG with Structured and Unstructured Knowledge Bases using Strands Agents

This notebook demonstrates how to build a RAG system that intelligently routes queries between structured and unstructured knowledge bases using Strands Agents with custom retrieve tools.

In [1]:
import os
import boto3
from strands import Agent, tool

In [2]:
UNSTRUCTURED_KB_ID = "FJ6OIBKNMP"  # Your document-based knowledge base
STRUCTURED_KB_ID = "G5F1E2PFCI"   # Your SQL/structured data knowledge base

In [3]:
# Set up AWS region and client
region = boto3.Session().region_name or "us-west-2"
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)

In [4]:
@tool
def unstructured_data_assistant(query: str) -> str:
    """
    Handle document-based, narrative, and conceptual queries using the unstructured knowledge base.
    
    Args:
        query: A question about business strategies, policies, company information, 
               or requiring document comprehension and qualitative analysis
    
    Returns:
        Raw retrieve response from the unstructured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=UNSTRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in unstructured data assistant: {str(e)}"


In [5]:
@tool
def structured_data_assistant(query: str) -> str:
    """
    Handle data analysis, metrics, and quantitative queries using the structured knowledge base.
    
    Args:
        query: A question requiring calculations, aggregations, statistical analysis,
               or database operations on structured data
    
    Returns:
        Raw retrieve response from the structured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=STRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in structured data assistant: {str(e)}"


As an intelligent assistant, your role is to route queries to one of two specialized knowledge bases. The unstructured data assistant handles questions about business strategies, policies, company information, culture, and qualitative topics, while the structured data assistant manages questions requiring calculations, metrics, financial data, and quantitative analysis. When you receive a query, you'll direct it to the appropriate tool - sending document, policy, and strategy questions to the unstructured assistant, and data, financial, and calculation questions to the structured assistant. Once these tools provide their raw data, you'll analyze it and present the information in a clear, helpful format for the user.



In [ ]:
# Create the orchestrator agent with both tools
orchestrator = Agent(
    system_prompt="""You are an intelligent assistant that routes queries to the appropriate knowledge base. Choose the appropriate tool based on the query type. 
    The tools return raw data that you should analyze and present in a clear, helpful format.""",
    tools=[
        unstructured_data_assistant,
        structured_data_assistant
    ]
)

In [ ]:
# EXAMPLE 1: Business Strategy Query (should use unstructured_data_assistant)
print("=== EXAMPLE 1: BUSINESS STRATEGY QUERY ===")
print("Query: What is Octank Financial's business strategy?")
print()

response = orchestrator("What is Octank Financial's business strategy?")
print(response)

=== TEST 1: BUSINESS STRATEGY QUERY ===
Query: What is Octank Financial's business strategy?

I'll help you find information about Octank Financial's business strategy. This is a question about company information that would be best answered using the unstructured knowledge base.
Tool #1: unstructured_data_assistant
# Octank Financial's Business Strategy

Based on the information retrieved, here's an overview of Octank Financial's business strategy:

## Core Business Focus
Octank Financial is a leading financial services company founded in 2010 that provides a comprehensive range of financial services including:
- Investment banking (mergers & acquisitions, capital markets, financial advisory)
- Wealth management for high-net-worth individuals and families
- Asset management for institutional clients (pension funds, endowments, foundations)
- Corporate finance for middle-market companies
- Private equity investments in growth-oriented companies

## Strategic Pillars

### 1. Client-Cent

In [ ]:
# EXAMPLE 2: Financial Data Query (should use structured_data_assistant)
print("=== EXAMPLE 2: FINANCIAL DATA QUERY ===")
print("Query: What is the total spending by all customers?")
print()

response = orchestrator("What is the total spending by all customers?")
print(response)


=== TEST 2: FINANCIAL DATA QUERY ===
Query: What is the total spending by all customers?

I'll help you find information about the total spending by all customers. This question requires calculating or retrieving specific financial data, so I'll use the structured data assistant.
Tool #2: structured_data_assistant
# Total Customer Spending

Based on the data retrieved from the structured database, the total spending by all customers at Octank Financial is:

**$5,078,473.69**

This figure represents the sum of all order totals in the company's order database.# Total Customer Spending

Based on the data retrieved from the structured database, the total spending by all customers at Octank Financial is:

**$5,078,473.69**

This figure represents the sum of all order totals in the company's order database.



In [10]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "What is Octank Financial's business strategy?"}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find information about Octank Financial's business strategy. This is a question about company information that would be best answered using the unstructured knowledge base."},
   {'toolUse': {'toolUseId': 'tooluse_23AN_a_lRcy6XpsA02zX1A',
     'name': 'unstructured_data_assistant',
     'input': {'query': "What is Octank Financial's business strategy?"}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_23AN_a_lRcy6XpsA02zX1A',
     'status': 'success',
     'content': [{'text': '{\'ResponseMetadata\': {\'RequestId\': \'f3ca7308-b0d7-4eb8-90e1-90643beaed4c\', \'HTTPStatusCode\': 200, \'HTTPHeaders\': {\'date\': \'Fri, 20 Jun 2025 20:22:05 GMT\', \'content-type\': \'application/json\', \'content-length\': \'19781\', \'connection\': \'keep-alive\', \'x-amzn-requestid\': \'f3ca7308-b0d7-4eb8-90e1-90643beaed4c\'}, 